# Data Science Aplicada à Área de Saúde

## Prevendo resultado de exame CTG

### por Antonildo Santos

In [ ]:
from IPython.display import Image
Image("../input/ctgimage/aparelho-ctg.png")

O objetivo deste trabalho é analisar um conjunto de dados contendo medições extraídas de cardiotocogramas fetais (CTGs) e criar um modelo para classificar o resultado do exame de Cardiotocograma (CTG) (que representa o bem-estar do feto).

## Introdução

A ausência de acompanhamento médico é considerada uma das principais causas de mortalidade infantil, conforme dados do Fundo de População das Nações Unidas (Fnuap), a taxa de mortalidade infantil mundial é de 45 óbitos a cada mil crianças nascidas vivas. Apesar de comprovadamente este número está em constante declínio é importante destacar que essa redução não ocorre na mesma proporção em todos os países. A ONU espera que, até 2030, os países acabem com as mortes evitáveis ​​de recém-nascidos e crianças menores de 5 anos de idade, e espera que todos os países reduzam a mortalidade de menores de 5 anos para pelo menos 25 por 1.000 nascidos vivos.

A morbimortalidade materna e perinatal continuam ainda muito elevadas no Brasil, incompatíveis com o atual nível de desenvolvimento econômico e social do País. Sabe-se que a maioria das mortes e complicações que surgem durante a gravidez,
parto e puerpério são preveníveis, mas para isso é necessária a participação ativa do sistema de saúde. 

Diante do exposto, os Cardiotocogramas (CTGs) são uma opção simples e de baixo custo para avaliação da saúde fetal, permitindo aos profissionais de saúde atuarem na prevenção da mortalidade infantil e materna. O próprio equipamento funciona enviando pulsos de ultrassom e lendo sua resposta, lançando luz sobre a frequência cardíaca fetal (FCF), movimentos fetais, contrações uterinas e muito mais. 
Hospitais, maternidades e clínicas obstétricas que buscam oferecer um atendimento integral à gestante devem estar preparados para a realização da cardiotocografia (CTG). O exame avalia a vitalidade do bebê e indica o sofrimento fetal, trazendo alertas como a necessidade da antecipação do parto, por exemplo.

### Sobre o conjunto de dados

Este conjunto de dados possui 2126 medições extraídas de cardiotocogramas fetais (CTGs) que foram processados ​​automaticamente e os respectivos recursos diagnósticos medidos. Os CTGs também foram classificados por três obstetras especialistas e uma etiqueta de classificação de consenso atribuída a cada um deles, conforme descrito em:

Ayres de Campos et al. (2000) SisPorto 2.0 A Program for Automated Analysis of Cardiotocograms. J Matern Fetal Med 5: 311-318

Fonte dos dados: https://archive.ics.uci.edu/ml/datasets/cardiotocography

### Informações sobre os atributos:

    baseline value                                         - linha de base FHR (batimentos por minuto)
    accelerations                                          - número de acelerações por segundo
    fetal_movement                                         - número de contrações uterinas por segundo
    light_decelerations                                    - número de desacelerações leves por segundo
    severe_decelerations                                   - número de desacelerações graves por segundo
    prolongued_decelerations                               - número de desacelerações prolongadas por segundo
    abnormal_short_term_variability                        - porcentagem de tempo com variabilidade anormal de curto prazo
    mean_value_of_short_term_variability                   - valor médio de variabilidade de curto prazo
    percentage_of_time_with_abnormal_long_term_variability - porcentagem de tempo com variabilidade anormal de longo prazo
    mean_value_of_long_term_variability                    - valor médio de variabilidade de longo prazo
    histogram_width                                        - largura do histograma FHR
    histogram_min                                          - mínimo do histograma FHR
    histogram_max                                          - Máximo do histograma FHR
    histogram_number_of_peaks                              - Nº de picos do histograma
    histogram_number_of_zeroes                             - Nº de zeros do histograma
    histogram_mode                                         - modo histograma
    histogram_mean                                         - média do histograma
    histogram_median                                       - mediana do histograma
    histogram_variance                                     - variância do histograma
    histogram_tendency                                     - tendência do histograma
    fetal_health                                           - código de classe do estado fetal 
                                                            (1 = normal; 2 = suspeito; 3 = patológico)

In [ ]:
# Carregando os pacotes
import pandas as pd
import numpy as np
import statsmodels.api as sm
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectFromModel
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import cohen_kappa_score
from sklearn.ensemble import RandomForestClassifier, ExtraTreesRegressor, ExtraTreesClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.feature_selection import RFECV
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score, roc_curve, auc, precision_score, recall_score, roc_auc_score 
import warnings
warnings.filterwarnings("ignore")

In [ ]:
pd.__version__

In [ ]:
np.__version__

## Carregando os Dados

In [ ]:
fetal_health = pd.read_csv('../input/fetal-health-classification/fetal_health.csv')

## Análise Exploratória

In [ ]:
fetal_health.head(20)

In [ ]:
fetal_health.info()

In [ ]:
# Verificando valores nulos
fetal_health.isnull().sum()

In [ ]:
fetal_health['fetal_health'] = fetal_health['fetal_health'].astype('int64')

Em nosso conjunto de dados não existem dados faltantes, isto é muito bom, pois não precisaremos utilizar nenhum tipo de método de inputação de dados.

In [ ]:
# Formato dos dados
fetal_health.shape

In [ ]:
# Verificando valores únicos
for col in list(fetal_health.columns):
    
    # Obtém uma lista de valores únicos
    list_of_unique_values = fetal_health[col].unique()
    
    # Se o número de valores exclusivos for menor que 15, imprima os valores. 
    # Caso contrário, imprima o número de valores exclusivos
    if len(list_of_unique_values) < 15:
        print("\n")
        print(col + ': ' + str(len(list_of_unique_values)) + ' valores únicos')
        print(list_of_unique_values)
    else:
        print("\n")
        print(col + ': ' + str(len(list_of_unique_values)) + ' valores únicos')

In [ ]:
# Checando as colunas que tem valor = '?'
fetal_health.isin(['?']).any()

In [ ]:
fetal_health['fetal_health'].value_counts()

In [ ]:
sns.set(style="whitegrid")

#Usando um gráfico de barras para mostrar a distribuição das classes: Morrer e Sobreviver
bp = sns.countplot(x=fetal_health['fetal_health'])
plt.title("Distribuição de classe do conjunto de dados")
bp.set_xticklabels(["Normal","Suspeito","Patológico"])
plt.show()

In [ ]:
# Verifica a proporção de cada classe
round(fetal_health['fetal_health'].value_counts() / len(fetal_health.index) * 100, 0)

In [ ]:
# Vamos visualizar de forma gráfica

# Percentual de cada valor da variável alvo
percentual = round(fetal_health['fetal_health'].value_counts() / len(fetal_health.index) * 100, 0)

# Labels
labels = ["Normal","Suspeito","Patológico"]

# Plot
plt.axis("equal")
plt.pie(percentual , 
        labels = labels,
        radius = 1.6,
        autopct = '%1.2f%%',
        explode = [0.05,0.05,0.05],
        startangle = 90,
        shadow = True,
        counterclock = False,
        pctdistance = 0.6)
plt.show()

In [ ]:
# Coletando estatísticas das colunas
fetal_health.describe()

In [ ]:
# Função para visualizar a distribuição de cada variável
def cria_histograma(df, features, rows, cols):
    fig = plt.figure(figsize = (20,20))
    
    for i, feature in enumerate(features):
        ax = fig.add_subplot(rows, cols, i+1)
        df[feature].hist(bins = 20, ax = ax, facecolor = 'midnightblue')
        ax.set_title(feature + " Distribuição", color = 'DarkRed')
        
    fig.tight_layout()  
    plt.show()

In [ ]:
# Executa a função
cria_histograma(fetal_health, fetal_health.columns, 8, 3)

In [ ]:
# Avaliando a correlação das variáveis independentes com a variável alvo
corr = fetal_health.corr()
corr[['fetal_health']].sort_values(by = 'fetal_health',ascending = False).style.background_gradient()

In [ ]:
fetal_health_clean = fetal_health.drop('fetal_health', axis = 1)
y = fetal_health['fetal_health']

In [ ]:
fetal_health_clean

In [ ]:
# Avaliando a Correlação das variáveis
corr = fetal_health_clean.corr()

In [ ]:
# Cria o mapa de calor com a matriz de correlação
f, ax = plt.subplots(figsize = (15, 9))
sns.heatmap(corr, cbar = True, annot = True, fmt = '.2f', annot_kws = {'size': 10}, vmax = 1, square = True, cmap = 'rainbow')
plt.show()

Através do gráfico de correlação podemos perceber que existe colinearidade entre algumas variáveis, como por exemplo histogram_median, histogram_mean, histogram_mode. Vamos constatar isso através do metodo VIF (Variance Inflation Factor).

In [ ]:
# Import library for VIF
from statsmodels.stats.outliers_influence import variance_inflation_factor

def calc_vif(X):

    # Calculating VIF
    vif = pd.DataFrame()
    vif["variables"] = X.columns
    vif["VIF"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]

    return(vif)

In [ ]:
calc_vif(fetal_health_clean)

VIF superior a 5 ou 10 indica alta multicolinearidade entre esta variável independente e as outras, neste caso podemos perceber uma alta colinearidade entre as variáveis histogram_mode, histogram_mean, histogram_median, baseline value, também entre as variáveis histogram_width, histogram_mean e histogram_median.

Resolverei o problema de multicolinearidade empregando a análise fatorial para agrupar em fatores as variávies com alta colinearidade, depois eliminarei as variáveis colineares, mantendo as demais variáveis juntamente com os fatores.

In [ ]:
fetal_health_clean.shape[1]

### Aplicando transformação nos dados

In [ ]:
# Renomeando a variável baseline value para baseline_value
fetal_health_clean['baseline_value'] = fetal_health_clean['baseline value']
del fetal_health_clean['baseline value']

In [ ]:
!pip install factor_analyzer

In [ ]:
from sklearn.decomposition import FactorAnalysis
from factor_analyzer import FactorAnalyzer
from factor_analyzer.factor_analyzer import calculate_bartlett_sphericity
from factor_analyzer.factor_analyzer import calculate_kmo

In [ ]:
# 
scaler = StandardScaler()

scaled_fetal_health_clean=fetal_health_clean.copy()
scaled_fetal_health_clean=pd.DataFrame(scaler.fit_transform(scaled_fetal_health_clean), columns=scaled_fetal_health_clean.columns)
scaled_fetal_health_clean.info()

### Teste de Adequação para Análise Fatorial
Antes de realizar a análise fatorial, precisamos avaliar a “fatorabilidade” de nosso conjunto de dados. Fatorabilidade significa "podemos encontrar os fatores no conjunto de dados?". Existem dois métodos para verificar a fatorabilidade ou adequação da amostragem:

* Teste de Bartlett
* Teste Kaiser-Meyer-Olkin

O teste de esfericidade de Bartlett verifica se as variáveis ​​observadas estão correlacionadas entre si ou não, usando a matriz de correlação observada contra a matriz de identidade. Se o teste não for estatisticamente significante, não devemos empregar uma análise fatorial.

O teste Kaiser-Meyer-Olkin (KMO) mede a adequação dos dados para a análise fatorial. Ele determina a adequação para cada variável observada e para o modelo completo. KMO estima a proporção da variância entre todas as variáveis ​​observadas. Menor proporção é mais adequado para análise fatorial. Os valores de KMO variam entre 0 e 1. O valor de KMO menor que 0,6 é considerado inadequado.

In [ ]:
#Teste de adequação

#Bartlett
#p-value should be 0 (statistically sig.)
chi_square_value,p_value=calculate_bartlett_sphericity(scaled_fetal_health_clean)
print(chi_square_value, p_value)

In [ ]:
#KMO
#Value should be 0.6<
kmo_all,kmo_model=calculate_kmo(scaled_fetal_health_clean)
print(kmo_model)

O teste de Bartlett retornou o valor p igual zero, isso quer dizer que o teste foi estatisticamente significativo, indicando que a matriz de correlação observada não é uma matriz identidade.

O KMO geral para nossos dados é de 0.74, o que é excelente. Este valor indica que posso prosseguir com a análise fatorial planejada.

In [ ]:
scaled_fetal_health_clean

### Escolhendo o número de fatores

Para escolher o número de fatores, você pode usar o critério de Kaiser e o gráfico de scree. Ambos são baseados em valores próprios.

In [ ]:
#EXPLORATORY FACTOR ANALYSIS
fa = FactorAnalyzer(rotation=None)
fa.fit(scaled_fetal_health_clean)

In [ ]:
#GET EIGENVALUES
ev, v = fa.get_eigenvalues()

In [ ]:
# SCREEPLOT (need pyplot)
plt.scatter(range(1,scaled_fetal_health_clean.shape[1]+1),ev)
plt.plot(range(1,scaled_fetal_health_clean.shape[1]+1),ev)
plt.title('Scree Plot')
plt.xlabel('Factors')
plt.ylabel('Eigenvalue')
plt.grid(True)
plt.show()

O método do scree plot desenha uma linha reta para cada fator e seus autovalores. Numere os valores próprios maiores que um considerado como o número de fatores.

Aqui, você pode ver apenas que os valores próprios de 5 fatores são maiores que um. Isso significa que precisamos escolher apenas 5 fatores (ou variáveis ​​não observadas). Porém após avaliar as possibilidades cheguei ao número de 3 fatores.

In [ ]:
fa = FactorAnalysis(n_components=3, random_state=0, svd_method='lapack')

In [ ]:
fa.fit(scaled_fetal_health_clean)

In [ ]:
# Create factor analysis object and perform factor analysis
fa = FactorAnalyzer(3, rotation="varimax", method='minres', use_smc=True)
fa.fit(scaled_fetal_health_clean)

In [ ]:
fa.loadings_

In [ ]:
fa.get_communalities()

In [ ]:
# Get variance of each factors
fa.get_factor_variance()

In [ ]:
loadings = pd.DataFrame(fa.loadings_, columns=['Factor 1', 'Factor 2', 'Factor 3'], index=scaled_fetal_health_clean.columns)
fator1 = loadings['Factor 1'].copy()
fator2 = loadings['Factor 2'].copy()
fator3 = loadings['Factor 3'].copy()
print('Factor 1 \n%s' %fator1.sort_values(ascending=False))
print('')
print('Factor 2 \n%s' %fator2.sort_values(ascending=False))
print('')
print('Factor 3 \n%s' %fator3.sort_values(ascending=False))

O fator 1 tem altas cargas fatoriais para:
* histogram_width
* histogram_variance
* histogram_number_of_peaks
* mean_value_of_short_term_variability
* histogram_max
* light_decelerations

O fator 2 tem altas cargas fatoriais para:
* histogram_mode
* histogram_mean
* histogram_median
* baseline_value

O fator 3 tem altas cargas fatoriais para:
* abnormal_short_term_variability
* percentage_of_time_with_abnormal_long_term_variability


O Cronbach Alfa pode ser usado para medir se as variáveis de um fator formam ou não um fator “coerente” e confiável. Um valor acima de 0.6 para o alfa é, na prática, considerado aceitável.

In [ ]:
!pip install pingouin

In [ ]:
import pingouin as pg

In [ ]:
#Create the factors
factor1 = scaled_fetal_health_clean[['histogram_width', 'histogram_variance', 'histogram_number_of_peaks', 'mean_value_of_short_term_variability', 'histogram_max', 'light_decelerations']]
factor2 = scaled_fetal_health_clean[['histogram_mode', 'histogram_mean', 'histogram_median', 'baseline_value']]
factor3 = scaled_fetal_health_clean[['abnormal_short_term_variability', 'percentage_of_time_with_abnormal_long_term_variability']]
#Get cronbach alpha
factor1_alpha = pg.cronbach_alpha(factor1)
factor2_alpha = pg.cronbach_alpha(factor2)
factor3_alpha = pg.cronbach_alpha(factor3)
print(factor1_alpha, factor2_alpha, factor3_alpha)

Os alfas são avaliados em 0.87, 0.95 e 0.69, o que indica que eles são úteis e coerentes.

In [ ]:
# Criando um Dataframe com apenas com as colunas que possuem maior relevência para os fatores 
scaled_fetal_health_fact = pd.concat([factor1, factor2, factor3], axis=1, join='inner')

In [ ]:
scaled_fetal_health_fact

In [ ]:
# Aplicando os fatores ao Dataframe
scaled_fetal_health_new = fa.fit_transform(scaled_fetal_health_fact)

In [ ]:
coluns_fact = ['factor1', 'factor2', 'factor3']

In [ ]:
scaled_fetal_health_new_fact=pd.DataFrame(scaled_fetal_health_new, columns=coluns_fact)

In [ ]:
fetal_health_clean_new = fetal_health_clean.copy()

In [ ]:
# Eliminando a colunas com colinearidade que foram substituidas por fatores
for col in factor1:
    del fetal_health_clean_new[col]
for col in factor2:
    del fetal_health_clean_new[col]
for col in factor3:
    del fetal_health_clean_new[col]

In [ ]:
# Criando a versão final do Dataframe 
fetal_health_clean_finish = pd.concat([fetal_health_clean_new, scaled_fetal_health_new_fact], axis=1, join='inner')

In [ ]:
fetal_health_clean_finish

In [ ]:
fetal_health_clean_finish.shape

In [ ]:
fetal_health_clean_finish.columns

In [ ]:
X = fetal_health_clean_finish

In [ ]:
# Aplica a divisão com proporção 70/30
X_treino, X_teste, y_treino, y_teste = train_test_split(X, y, test_size = 0.30, random_state = 101)

In [ ]:
# Treina o padronizador com o método fit() e aplica com o método transform() nos dados de treino e teste.

X_scaled_treino = X_treino.copy()
X_scaled_teste = X_teste.copy()

# features
num_cols = fetal_health_clean_finish.columns

# Padronizando as variáveis de entrada
for i in num_cols:
    
    # fit on training data column
    scale = StandardScaler().fit(X_scaled_treino[[i]])
    
    # transform the training data column
    X_scaled_treino[i] = scale.transform(X_scaled_treino[[i]])
    X_scaled_teste[i] = scale.transform(X_scaled_teste[[i]])

### Seleção de Atributos (Feature Selection)

In [ ]:
# Cria o seletor de variáveis

# Cria o estimador
estimador_rfc = RandomForestClassifier(random_state = 101)

# Cria o seletor
seletor_f1 = RFECV(estimator = estimador_rfc, step = 1, cv = StratifiedKFold(10), scoring='f1_macro')

# Treinamos o seletor
seletor_f1 = seletor_f1.fit(X_scaled_treino, y_treino)

In [ ]:
print('Número Ideal de Atributos: {}'.format(seletor_f1.n_features_))

In [ ]:
# Vamos avaliar a acurácia do modelo com F1 Score
previsoes_seletor_f1 = seletor_f1.predict(X_scaled_teste)
from sklearn.metrics import accuracy_score
acc_seletor_f1 = accuracy_score(y_teste, previsoes_seletor_f1)
acc_seletor_f1

In [ ]:
# Visualiza os scores das variáveis mais importantes
seletor_f1.estimator_.feature_importances_

In [ ]:
X_scaled_treino.columns

In [ ]:
seletor_f1.support_

In [ ]:
X_scaled_treino# Cria um dataframe com os resultados
resultado_seletor_f1 = pd.DataFrame()
resultado_seletor_f1['Atributo'] = X_scaled_treino.columns[np.where(seletor_f1.support_ == True)]
resultado_seletor_f1['Score'] = seletor_f1.estimator_.feature_importances_
resultado_seletor_f1.sort_values('Score', inplace = True, ascending = True)

In [ ]:
# Plot 
#plt.figure(figsize = (10, 10))
plt.barh(y = resultado_seletor_f1['Atributo'], width = resultado_seletor_f1['Score'], color = 'Blue')
plt.title('Importância de Variáveis - RFECV', fontsize = 18, fontweight = 'bold', pad = 10)
plt.xlabel('Importância', fontsize = 14, labelpad = 15)
plt.show()

In [ ]:
# Extrai as variáveis e quais são importante ou não para o modelo
variaveis_rfecv = pd.Series(seletor_f1.support_, index = X_scaled_treino.columns)
variaveis_rfecv

In [ ]:
del X_scaled_treino['severe_decelerations']
del X_scaled_treino['histogram_number_of_zeroes']
del X_scaled_treino['histogram_tendency']
#del X_scaled_treino['fetal_movement']

In [ ]:
del X_scaled_teste['severe_decelerations']
del X_scaled_teste['histogram_number_of_zeroes']
del X_scaled_teste['histogram_tendency']
#del X_scaled_teste['fetal_movement']

In [ ]:
# Fit model using each importance as a threshold
thresholds = np.sort(seletor_f1.estimator_.feature_importances_)
for thresh in thresholds:
    # select features using threshold
    selection = SelectFromModel(seletor_f1.estimator_, threshold=thresh, prefit=True)
    select_X_treino = selection.transform(X_scaled_treino)
    # train model
    selection_model = ExtraTreesClassifier()
    selection_model.fit(select_X_treino, y_treino)
    # eval model
    select_X_teste = selection.transform(X_scaled_teste)
    y_pred = selection_model.predict(select_X_teste)
    predictions = [round(value) for value in y_pred]
    accuracy = accuracy_score(y_teste, predictions)
    print("Thresh=%.3f, n=%d, Accuracy: %.2f%%" % (thresh, select_X_treino.shape[1], accuracy*100.0))

## Criando o Classificador

In [ ]:
# Regressão Logística

# Cria o modelo
modelo_lr = LogisticRegression(tol = 1e-7, penalty = 'l2', C = 0.1, solver = 'liblinear', multi_class='ovr')

# Treina o modelo
modelo_lr.fit(X_scaled_treino, y_treino)

# Faz as previsões
y_pred = modelo_lr.predict(X_scaled_teste)
predict_proba = modelo_lr.predict_proba(X_scaled_teste)

print(confusion_matrix(y_teste, y_pred))

# Métricas Globais
cohen_kappa_lr = cohen_kappa_score(y_teste, y_pred)
acc_lr = modelo_lr.score(X_scaled_teste, y_teste)
print("\nCoeficiente Cohen kappa = {}".format(cohen_kappa_lr))
print("Acurácia = {}".format(acc_lr))
print("")

# Relatório de Classificação
print(classification_report(y_teste, y_pred, target_names = ['Normal', 'Suspeito','Patológico']))

In [ ]:
# XGBoost

import xgboost as xgb

# Cria o modelo
xgb_model = xgb.XGBClassifier (eta=0.1,
                               max_depth=3,
                               min_child_weight=8,
                               learning_rate=0.1, 
                               colsample_bytree = 0.8,
                               subsample = 0.80,
                               objective='multi: softprob',
                               n_estimators=65,
                               reg_alpha = 0.01,
                               num_class=3,
                               gamma=0.01,
                               random_state=42)
# Treina o modelo
xgb_model.fit(X_scaled_treino, y_treino)

# Faz as previsões
y_pred = xgb_model.predict(X_scaled_teste)
predict_proba = xgb_model.predict_proba(X_scaled_teste)

print(confusion_matrix(y_teste, y_pred))

# Métricas Globais
cohen_kappa_xgb = cohen_kappa_score(y_teste, y_pred)
acc_xgb = xgb_model.score(X_scaled_teste, y_teste)
print("\nCoeficiente Cohen kappa = {}".format(cohen_kappa_xgb))
print("Acurácia = {}".format(acc_xgb))
print("")

# Relatório de Classificação
print(classification_report(y_teste, y_pred, target_names = ['Normal', 'Suspeito','Patológico']))

In [ ]:
# RandomForestClassifier

# Cria o modelo
RFclf=RandomForestClassifier(n_estimators=100, random_state=42)

# Treina o modelo
RFclf.fit(X_scaled_treino, y_treino)

# Faz as previsões
y_pred=RFclf.predict(X_scaled_teste)
predict_proba = RFclf.predict_proba(X_scaled_teste)

print(confusion_matrix(y_teste, y_pred))

# Métricas Globais
cohen_kappa_RF = cohen_kappa_score(y_teste, y_pred)
acc_RF = RFclf.score(X_scaled_teste, y_teste)
print("\nCoeficiente Cohen kappa = {}".format(cohen_kappa_RF))
print("Acurácia = {}".format(acc_RF))
print("")

# Relatório de Classificação
print(classification_report(y_teste, y_pred, target_names = ['Normal', 'Suspeito','Patológico']))

In [ ]:
# Criando o classificador ExtraTreesClassifier
ETclf=ExtraTreesClassifier(n_estimators=100,random_state=42)

#Train the model 
ETclf.fit(X_scaled_treino, y_treino)

# prediction on test set
y_pred=ETclf.predict(X_scaled_teste)
predict_proba = ETclf.predict_proba(X_scaled_teste)

print(confusion_matrix(y_teste, y_pred))

# Métricas Globais
cohen_kappa_ET = cohen_kappa_score(y_teste, y_pred)
acc_ET = ETclf.score(X_scaled_teste, y_teste)
print("\nCoeficiente Cohen kappa = {}".format(cohen_kappa_ET))
print("Acurácia = {}".format(acc_ET))
print("")

# Relatório de Classificação
print(classification_report(y_teste, y_pred, target_names = ['Normal', 'Suspeito','Patológico']))

Após comparar o desempenho entre os modelos, o que obteve a melhor performance foi o RandomForestClassifier, agora vamos avaliar a sua performance.

In [ ]:

y_pred=RFclf.predict(X_scaled_teste)
predict_proba = RFclf.predict_proba(X_scaled_teste)
conf_matriz = confusion_matrix(y_teste, y_pred)

## Avaliando a performance do modelo

In [ ]:
TP = conf_matriz[0,0] + conf_matriz[1,1] + conf_matriz[2,2]
TN = conf_matriz[0,1] + conf_matriz[0,2] + conf_matriz[1,0] + conf_matriz[1,2] + conf_matriz[2,0] + conf_matriz[2,2]
FP = conf_matriz[0,1] + conf_matriz[0,2] + conf_matriz[1,0] + conf_matriz[1,2] + conf_matriz[2,0] + conf_matriz[2,2]
FN = conf_matriz[1,0] + conf_matriz[2,0] + conf_matriz[0,1] + conf_matriz[2,1] + conf_matriz[0,2] + conf_matriz[1,2]

In [ ]:
# classe Normal
TP = conf_matriz[0,0]
TN = conf_matriz[1,1] + conf_matriz[1,2] + conf_matriz[2,1] + conf_matriz[2,2] 
FN = conf_matriz[0,1] + conf_matriz[0,2]
FP = conf_matriz[1,0] + conf_matriz[2,0]
Acc = (TP + TN) / (TP + TN + FP + FN) 
precisao = TP / (TP + FP) 
sensibilidade = TP / (TP + FN)
especificidade = TN / (TN + FP)
Pontuação_F = 2 * TP / (2 * TP + FP + FN)
#
# Print
print('\nClasse Normal')
print('Precisão :', precisao)
print('sensibilidade :', sensibilidade)
print('Especificidade :', especificidade)
print('Pontuação_F :', Pontuação_F)

In [ ]:
# classe Suspeito
TP = conf_matriz[1,1]
TN = conf_matriz[0,0] + conf_matriz[1,2] + conf_matriz[2,1] + conf_matriz[2,2] 
FN = conf_matriz[1,0] + conf_matriz[1,2]
FP = conf_matriz[0,1] + conf_matriz[2,1]
Acc = (TP + TN) / (TP + TN + FP + FN) 
precisao = TP / (TP + FP) 
sensibilidade = TP / (TP + FN)
especificidade = TN / (TN + FP)
Pontuação_F = 2 * TP / (2 * TP + FP + FN)
#
# Print
print('\nClasse Suspeito')
print('Precisão :', precisao)
print('sensibilidade :', sensibilidade)
print('Especificidade :', especificidade)
print('Pontuação_F :', Pontuação_F)

In [ ]:
# classe Patológico
TP = conf_matriz[2,2]
TN = conf_matriz[0,0] + conf_matriz[0,1] + conf_matriz[1,0] + conf_matriz[1,1] 
FP = conf_matriz[2,0] + conf_matriz[2,1]
FN = conf_matriz[0,2] + conf_matriz[1,2]
precisao = TP / (TP + FP) 
sensibilidade = TP / (TP + FN)
especificidade = TN / (TN + FP)
Pontuação_F = 2 * TP / (2 * TP + FP + FN)
#
# Print
print('\nClasse Patológico')
print('Precisão :', precisao)
print('sensibilidade :', sensibilidade)
print('Especificidade :', especificidade)
print('Pontuação_F :', Pontuação_F)

In [ ]:
# Relatório de Classificação
print(classification_report(y_teste, y_pred, target_names = ['Normal', 'Suspeito','Patológico']))

In [ ]:
import scikitplot as skplt
skplt.metrics.plot_roc(y_teste, predict_proba, figsize=(10, 8))

### Interpretando o resultado do Modelo
Para realizar este trabalho utilizei o pacote LIME, que serve para gerar explicações locais para o modelo. A ideia central por trás da técnica é bastante intuitiva. Suponha que temos um classificador complexo, com um limite de decisão altamente não linear, seu objetivo é entender por que o modelo de aprendizado de máquina fez uma determinada previsão. O LIME testa o que acontece com as previsões quando você dá variações de seus dados ao modelo de aprendizado de máquina.

In [ ]:
import lime
import lime.lime_tabular

In [ ]:
# LIME tem um explainer para todos os tipos de modelos
explainer_v1 = lime.lime_tabular.LimeTabularExplainer(X_scaled_treino.values,  
                              feature_names = X_scaled_treino.columns.values.tolist(), 
                              class_names = ['Normal', 'Suspeito','Patológico'],  
                              verbose = True, 
                              mode = 'classification')

In [ ]:
import cufflinks as cf
cf.go_offline()
cf.set_config_file(offline=False, world_readable=True)

#### Submodular Pick And Global Explanations
Serve para encontrar um grupo de interpretações que tentam explicar a maioria dos casos. Utilizei uma amostra de 190 registros, o que representa 30% dos dados de teste.

In [ ]:
from lime import submodular_pick
sp_obj = submodular_pick.SubmodularPick(explainer_v1, X_scaled_treino.values, RFclf.predict_proba, sample_size=190, num_features=9, num_exps_desired=5)
#Plot the 5 explanations
[exp.as_pyplot_figure(label=exp.available_labels()[0]) for exp in sp_obj.sp_explanations];
# Make it into a dataframe
W_pick=pd.DataFrame([dict(this.as_list(this.available_labels()[0])) for this in sp_obj.sp_explanations]).fillna(0)
 
W_pick['prediction'] = [this.available_labels()[0] for this in sp_obj.sp_explanations]
 
#Making a dataframe of all the explanations of sampled points
W=pd.DataFrame([dict(this.as_list(this.available_labels()[0])) for this in sp_obj.explanations]).fillna(0)
W['prediction'] = [this.available_labels()[0] for this in sp_obj.explanations]
W['prediction']  = W.apply(lambda row: 'Normal' if (row['prediction'] == 0) else row['prediction'], axis=1)
W['prediction']  = W.apply(lambda row: 'Suspeito' if (row['prediction'] == 1) else row['prediction'], axis=1)
W['prediction']  = W.apply(lambda row: 'Patológico' if (row['prediction'] == 2) else row['prediction'], axis=1)
#Plotting the aggregate importances
np.abs(W.drop("prediction", axis=1)).mean(axis=0).sort_values(ascending=False).head(25).sort_values(ascending=True).iplot(kind="barh")
 
#Aggregate importances split by classes
grped_coeff = W.groupby("prediction").mean()
 
grped_coeff = grped_coeff.T
grped_coeff["abs"] = np.abs(grped_coeff.iloc[:, 0])
grped_coeff.sort_values("abs", inplace=True, ascending=False)
grped_coeff.head(25).sort_values("abs", ascending=True).drop("abs", axis=1).iplot(kind="barh", bargap=0.05) 


O primeiro gráfico dá uma ideia de quais recursos são importantes para o modelo num sentido mais amplo. Bem no topo do gráfico, podemos encontrar o número de desacelerações prolongadas por segundo como um indicador muito forte para diagnosticar o estado fetal, seguido do "fator 3" que representa o valor médio de variabilidade de curto prazo e porcentagem de tempo com variabilidade anormal de longo prazo.

O segundo gráfico divide a inferência entre os três rótulos e os examina separadamente. Este gráfico nos permite entender qual recurso foi mais importante na previsão de uma classe específica. Numa visão geral podemos perceber que basicamente os mesmos recursos são importantes para todas as classes, porém para classe "Normal" o número de desacelerações prolongadas por segundo como o indicador mais forte, já para as classes "Suspeito" e "Patológico" destaca-se o "fator 3" que representa o valor médio de variabilidade de curto prazo e porcentagem de tempo com variabilidade anormal de longo prazo.

### Referências

Formação Inteligência Artificial Aplicada à Medicina
https://www.datascienceacademy.com.br/

Ministério da Saúde http://bvsms.saude.gov.br/bvs/publicacoes/manual_tecnico_gestacao_alto_risco.pdf

Introduction to Factor Analysis in Python https://www.datacamp.com/community/tutorials/introduction-factor-analysis

RFECV https://www.scikit-yb.org/en/latest/api/model_selection/rfecv.html

Interpretability part 3: opening the black box with LIME and SHAP https://www.kdnuggets.com/2019/12/interpretability-part-3-lime-shap.html


# Fim